In [7]:
#After running the code add music file from your computer or from melody folder.
#Select the music file to play. 
#Click on play button to play the song.


#Packages used in creating music player Gui

from tkinter import*
import os                                              #Used for accessing the file path of music files from operating system
import time                                             #Used for creating current time and total length format
import tkinter.messagebox                              #Used for creating messagebox
import threading                                      # Used to run multiple operations concurrently in the same process space.
from tkinter import filedialog                        #Used for selecting files from operating system and opening in the Gui
from tkinter import ttk                               #Predefined module used for accessing themes widget in tkinter
from ttkthemes import themed_tk as tk                 #Provide themes to music player gui
from pygame import mixer                              #Predefined module for doing different function in music player
from mutagen .mp3 import MP3                          #Used for playing MP3 files in the music player

#Themes 
root = tk.ThemedTk()
root.get_themes()                                  # Returns a list of all themes that can be set
root.set_theme("radiance")                         # Sets an available theme
mixer.init()

# Creating the statusbar 
statusbar=ttk.Label( root, text=" Welcome to Melody",relief=RAISED,anchor=W,font='Times 15 italic')
statusbar.pack(side=BOTTOM ,fill=X)

#Title of Music player 
root.title("Melody")
#Icon of music player
root.iconbitmap(r'C:\Users\91868\Desktop\melody\images\icon.ico')

mixer.init()                                       #initializing the mixer module 

#Functions 


playlist=[]


#Creating playlist box   
def add_to_playlist(filename):
    filename=os.path.basename(filename)
    index=0
    playlistbox.insert(index,filename)
    playlist.insert(index,filename_path)
    index+=1
    
def browse_file():
    global filename_path
    filename_path=filedialog.askopenfilename()
    add_to_playlist(filename_path)
    mixer.music.queue(filename_path)
    
#Creating  Menubar
menubar=Menu(root)
root.config(menu=menubar)
#create Submenu
subMenu=Menu(menubar,tearoff=0)
menubar.add_cascade(label="File" ,menu =subMenu)
subMenu.add_command(label="Open", command=browse_file)
subMenu.add_command(label="Exit" ,command =root.destroy)          #destroy the tkinter window
subMenu=Menu(menubar,tearoff=0)

def about_us():
    tkinter.messagebox.showinfo('About Melody','This is a music player created using Tkinter ')
    

menubar.add_cascade(label="Help",menu=subMenu)
subMenu.add_command(label="About us " ,command=about_us)
    
    

# Root Window - StatusBar, LeftFrame, RightFrame
# LeftFrame - The listbox (playlist)
# RightFrame - TopFrame,MiddleFrame and the BottomFrame

#Frames
leftframe=Frame(root)
leftframe.pack(side=LEFT,padx=10)

#Creating playlistbox
playlistbox=Listbox(leftframe)
playlistbox.pack()

def del_song():
    selected_song=playlistbox.curselection()
    selected_song=int(selected_song[0])
    playlistbox.delete([selected_song])
    playlist.pop()
    
#Button
#addind and deleting songs from playlist
addbtn1=ttk.Button(leftframe,text="Add",command=browse_file)
addbtn1.pack(side=LEFT)
delbtn2=ttk.Button(leftframe,text="Del",command=del_song)
delbtn2.pack(side=LEFT)

#Frames 
#Rightframe -Topframe , middleframe ,bottomframe
#Topframe- contains (Total length ,current time and text )
rightframe=Frame(root)
rightframe.pack()

topframe=Frame(rightframe)
topframe.pack()


lengthlabel=Label(topframe,text='Total length= --:--')
lengthlabel.pack(pady=10)
currenttimelabel =ttk.Label(topframe,text='Current time= --:--',relief=GROOVE)
currenttimelabel.pack(pady=10)

def show_details(play_song):
    file_data=os.path.splitext(play_song)
    if file_data[1]=='.mp3':
        audio=MP3(play_song)
        total_length=audio.info.length
    else:
        a=mixer.Sound(play_song)
        total_length=a.get_length()
        
        
    mins,secs=divmod(total_length,60)
    mins=round(mins)
    secs=round(secs)
    timeformat='{:02d}:{:02d}'.format(mins,secs)
    lengthlabel['text']="Total length"+ " "+timeformat
    t1=threading.Thread(target=start_count,args=(total_length,))       # threading
    t1.start()


#For calculating the current time
# mixer.music.get_busy(): - Returns FALSE when we press the stop button (music stop playing)
# Continue - Ignores all of the statements below it. We check if music is paused or not.
def start_count(t):
    global paused
    current_time = 0
    while current_time <= t and mixer.music.get_busy():
        if paused:
            continue
        else:
            mins, secs = divmod(current_time, 60)
            mins = round(mins)
            secs = round(secs)
            timeformat = '{:02d}:{:02d}'.format(mins, secs)
            currenttimelabel['text'] = "Current Time" + ' - ' + timeformat
            time.sleep(1)
            current_time += 1

#Frames
#Middleframe-contains(play ,pause and stop buttons)
middleframe=Frame(rightframe)
middleframe.pack(padx=30,pady=30)

#Playing the music
def play_music():
    global paused
    if paused:
        mixer.music.unpause()
        statusbar['text'] = "Music Resumed"
        paused = FALSE
    else:
        stop_music()
        time.sleep(1)
        selected_song = playlistbox.curselection()
        selected_song = int(selected_song[0])
        play_it = playlist[selected_song]
        mixer.music.load(play_it)
        mixer.music.play()
        statusbar['text'] = "Playing music" + ' - ' + os.path.basename(play_it)
        show_details(play_it)

            
         
           
#Stopping the music    
def stop_music():
    mixer.music.stop()
    statusbar['text']="Music Stopped"
    
#Pausing the music   
paused=FALSE
def pause_music():
    global paused
    paused=TRUE
    mixer.music.pause()
    statusbar['text']="Music Paused"




    
#Buttons
#ttk is proving themes to buttons(a module of themes package)
playPhoto=PhotoImage(file=r'C:\Users\91868\Desktop\melody\images\play.png')    
playBtn=ttk.Button(middleframe,image=playPhoto,command=play_music)  
playBtn.grid(row =0, column=0,padx=10 ,pady =10)
stopPhoto=PhotoImage(file=r'C:\Users\91868\Desktop\melody\images\stop.png')
stopBtn=ttk.Button(middleframe,image=stopPhoto,command=stop_music)
stopBtn.grid(row =0, column=1,padx=10 ,pady =10)
pausePhoto=PhotoImage(file= r'C:\Users\91868\Desktop\melody\images\pause.png ')
pauseBtn=ttk.Button(middleframe,image =pausePhoto , command=pause_music)
pauseBtn.grid(row =0, column=2,padx=10 ,pady =10)

#Frames
#Bottomframe-contains(rewind button ,mute button ,volume slider)
bottomframe=Frame(rightframe)
bottomframe.pack()

def set_vol(val):
    volume=float(val)/100
    mixer.music.set_volume(volume)      #set_volume of mixer takes value only from 0 to 1. Example - 0, 0.1,0.55,0.54.0.99,1
                                         
def rewind_music():
    play_music()
    statusbar['text']="Music Rewinded"
    
#Muting and unmuting the music    
muted=FALSE
def mute_music():
    global muted
    if muted:
        mixer.music.set_volume(0.5)
        volumeBtn.config(image=volumePhoto)
        scale.set(50)
        statusbar['text']="Music Unmuted"
        muted=FALSE
        
    else:
        mixer.music.set_volume(0)
        scale.set(0)
        volumeBtn.config(image=mutePhoto)
        statusbar['text']="Music Muted"
        muted=TRUE
        
#buttons(rewind and volume)
rewindPhoto=PhotoImage(file=r'C:\Users\91868\Desktop\melody\images\rewind.png')
rewindBtn=ttk.Button(bottomframe,image=rewindPhoto,command=rewind_music)
rewindBtn.grid(pady =20,row =1,column=0,padx=30)
scale=ttk.Scale(bottomframe,from_=0 ,to =100 ,orient= HORIZONTAL, command =set_vol)
scale.grid(row=1,column=3,padx=10)
scale.set(50)
mixer.music.set_volume(0.5)
mutePhoto=PhotoImage(file=r'C:\Users\91868\Desktop\melody\images\muted.png')
volumePhoto=PhotoImage(file=r'C:\Users\91868\Desktop\melody\images\volume-button.png')
volumeBtn=ttk.Button(bottomframe,image=volumePhoto,command=mute_music)
volumeBtn.grid (row=1,column=1,padx=10) 


#Stop music on closing the window 
#protocol function overwites the window closing functionality
def on_closing():
    stop_music()
    root.destroy()
    
                     

root.protocol("WM_DELETE_WINDOW",on_closing)
root.mainloop()
        